In [1]:
from seeq import spy
import numpy as np
import pandas as pd
import pylab as pl
import asyncio
import threading

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from PIL import Image

from bqplot import pyplot as bq
from bqplot import ColorScale

import ipywidgets as widgets
from ipywidgets import interactive,interact,Output

import functools

# from datetime import datetime,date,time,tzinfo
import datetime
import time

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from IPython.display import display,clear_output,HTML

from voila.app import Voila

In [2]:
def seeq_login():
    spy.login(credentials_file='../credentials_file.txt',
              url='https://explore.seeq.com',
              directory='Seeq', 
              ignore_ssl_errors=True)
    print('Login Successful')

In [3]:
def query_seeq_for_data():
    item1=spy.search({
            'Name': 'Temperature',
            'Path': 'Example >> Cooling Tower 1',
            'Type': 'StoredSignal'

             },all_properties=True)
    
    item2=spy.search({
        'Name':'Tower 1 Power Production',
        'Path': 'Example >> Cooling Tower 1',
        #     'Type': 'StoredSignal'
        },all_properties=True)
    
    return item1, item2

In [4]:
def pull_seeq_data(item1,item2,start_date,end_date):
    
    X_data=spy.pull(item1,start=start_date,end=end_date,grid='10min')
    Y_data=spy.pull(item2,start=start_date,end=end_date,grid='10min')
    
    return X_data,Y_data

In [5]:
def start_end_dummy_data():
    start_date='2020-01-01T12:00:00'
#     end_date='2020-01-10T12:00:00'
    end_date=datetime.datetime(2020,1,10,12,00,00)
    
    return start_date, end_date

[start_date,end_date]=start_end_dummy_data()

In [6]:
out=widgets.Output()

In [7]:
def train_test(X_data,Y_data):
    X_train,X_test,Y_train,Y_test=train_test_split(X_data,Y_data,test_size=0.6,random_state=0)
    
    return  X_train,X_test,Y_train,Y_test

In [8]:
def apply_PCA(X_data):
    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_data)
#     X_test=scaler.transform(X_test)
    
    apply_PCA.pca=PCA()
    apply_PCA.pca.fit(X_data)
    Xp_data=apply_PCA.pca.transform(X_data)
    
    components=apply_PCA.pca.components_
    explained_variance_ratio=apply_PCA.pca.explained_variance_ratio_
    explained_variance=apply_PCA.pca.explained_variance_[0:5]
    n_components=apply_PCA.pca.n_components_
    
    return Xp_data,components,explained_variance_ratio,explained_variance,n_components

In [9]:
def plot_PC_variance(X_data,explained_variance_ratio):
    Np_comp=(1,len(X_data.columns),1)
    @interact(layout={'width':'10px'})
    def pca_comp_var(Np=Np_comp):
        fig,ax=plt.subplots(1,1,figsize=(10,10))
        plt.figure(4)
        ax.set_xlabel('Principal Components')
        ax.set_ylabel('Explained Variance')

        ax.plot(np.arange(Np),explained_variance_ratio[0:Np],label='variance explained by each component')
        ax.bar(np.arange(Np),explained_variance_ratio[0:Np],label='Bar plot for explained var for each comp')
        ax.plot(np.arange(Np),np.cumsum(explained_variance_ratio[0:Np]),label='Cumulative variance explained')
        ax.legend(loc='upper left')

In [10]:
def plot_PC(X_data,components):
    Np_comp=(1,len(X_data.columns),1)
    @interact
    def principal_components(PC1=Np_comp,PC2=Np_comp):
        fig,ax=plt.subplots(1,1,figsize=(10,10))
        plt.figure(5)
        print(PC1, PC2)

        ax.set_xlabel("Principal Component {}".format(PC1), fontsize=14)
        ax.set_ylabel("Principal Component {}".format(PC2), fontsize=14)
        ax.set_title("Principal components {0} & {1}".format(PC1,PC2), fontsize=(20))
        ax.scatter(components[:,PC1],components[:,PC2])

In [11]:
def plot_tscore(X_data,Xp_data,components):
    Np_comp=(1,len(X_data.columns),1)
    t_score=np.matmul(Xp_data,components.T)
    len(t_score[:,2])
    @interact
    def t_scores(t1=Np_comp,t2=Np_comp):
        fig,ax=plt.subplots(1,1,figsize=(10,10))
        plt.figure(6)
        ax.set_title("t{0} & t{1} scores".format(t1,t2),fontsize=20)
        ax.set_xlabel("t{} score".format(t1),fontsize=14)
        ax.set_ylabel("t{} score".format(t2),fontsize=14)
        ax.scatter(t_score[:,t1],t_score[:,t2])
        
    return t_score

In [12]:
def plot_hotelling(X_data,t_score,explained_variance_ratio):
    Np_comp=(1,len(X_data.columns),1)
    print("Hotelling valid")
    
    @interact
    def hotelling(N=Np_comp):
        n=[np.square(t_score[:,i])/explained_variance_ratio[i] for i in range(N)]
        fig,ax=plt.subplots(1,1,figsize=(10,10))
        plt.figure(7)
        ax.set_title("Hotelling T2-First {} Components".format(N),fontsize=20)
        ax.set_xlabel("Time",fontsize=14)
        ax.set_ylabel("Hotelling Metric",fontsize=14)
        ax.plot(sum(n))

In [13]:
out2=widgets.Output()

In [14]:
def data_import_date(start_date,end_date):
    end_date1=end_date.strftime('%Y-%m-%dT%H:%M:%S')
    start_date=pd.Timestamp(start_date)
    end_date=pd.Timestamp(end_date1)

    button=widgets.Button(description='Pull Data')
    button.on_click(functools.partial(select_data,rs_=[start_date,end_date]))
    
    vbox=widgets.VBox([button])
    display(vbox,out)
    
def select_data(b,rs_):
#     clear_output()
    start_date=rs_[0]
    end_date1=rs_[1]
    print("Data pulling started")

    with out:
        clear_output(wait=True)
        seeq_login() 
        [item1,item2]=query_seeq_for_data()
        
        end_date=end_date1.strftime('%Y-%m-%dT%H:%M:%S')
        print("Start & End date: ",start_date,end_date)
        
     
        i=0    
        while True:
            end_date=end_date1.strftime('%Y-%m-%dT%H:%M:%S')

            if i==0:
                clear_output(wait=True)
                [X_data,Y_data]=pull_seeq_data(item1,item2,start_date,end_date)
                [Xp_data,components,explained_variance_ratio,_,_]=apply_PCA(X_data)
                plot_PC_variance(X_data,explained_variance_ratio)
                plot_PC(X_data,components)

            else:
                [X_data_live,Y_data_live]=pull_seeq_data(item1,item2,start_date,end_date)
                X_data=X_data.append(X_data_live)
                Y_data=Y_data.append(Y_data_live)
                Xp_data=apply_PCA.pca.transform(X_data)
                clear_output(wait=True)
                display(plot_PC_variance(X_data,explained_variance_ratio))
                display(plot_PC(X_data,components))

            print("Data pulling completed.\nNow you're ready for your analysis")

            t_score=plot_tscore(X_data,Xp_data,components)
            plot_hotelling(X_data,t_score,explained_variance_ratio)

            time.sleep(20)
            start_date=end_date
            end_date1=end_date1+datetime.timedelta(days=1)
            i+=1
            
a=interact(data_import_date,
           start_date=widgets.DatePicker(value=pd.to_datetime(start_date)),
           end_date=widgets.DatePicker(value=pd.to_datetime(end_date)))

interactive(children=(DatePicker(value=Timestamp('2020-01-01 12:00:00'), description='start_date'), DatePicker…

In [15]:
# import threading
# from IPython.display import display
# import ipywidgets as widgets
# import time
# progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

# def work(progress):
#     total = 100
#     for i in range(total):
#         time.sleep(0.2)
#         progress.value = float(i+1)/total

# thread = threading.Thread(target=work, args=(progress,))
# display(progress)
# thread.start()